# Game Extensive form to Normal form


<img src="Images/3.png">

## the input representation:
#### first line:
N(number of nodes) P(number of players) E(number of edges) L(number of leaf nodes) I(number of information sets) Na(number of nature nodes)
#### P following lines:
[nodes belong to player i]
#### E following lines:
source_node dest_node transition
#### L following lines:
leaf_node [value for each player]
#### I following lines:
[nodes in information set i]
#### Na following lines:
nature node i, number of transitions
[probability of each transition, destination of transition]


In [1]:
import numpy as np
N = P = E = L = I = Na = 0
player_nodes = []
node_master = []
child = []
utility = []
leaf_node = []
info_set = []
node_info_set = []
nature_node = []
nature_transition = []
result = []
header = []
done = []
first = []
count = 0

In [2]:
def read_input(path):
    f = open(path, 'r')
    global N, P, E, L, I, Na
    global player_nodes, node_master, child, utility, leaf_node, info_set, node_info_set 
    global nature_node, nature_transition
    N = P = E = L = I = Na = 0
    player_nodes = []
    node_master = []
    child = []
    utility = []
    leaf_node = []
    info_set = []
    node_info_set = []
    nature_node = []
    nature_transition = []
    N, P, E, L, I, Na = [int(i) for i in f.readline().split()]

    node_master = [-1 for i in range(N)]
    for i in range(P):
        player_nodes.append([int(j) for j in f.readline().split()])
        for j in player_nodes[i] : node_master[j] = i

    child = [[] for i in range(N)]
    for i in range(E):
        s, t, d = f.readline().split()
        child[int(s)].append((int(d), t))

    leaf_node = [-1 for i in range(N)]
    for i in range(L):
        inp = [int(i) for i in f.readline().split()]
        leaf_node[inp[0]] = i
        utility.append([inp[i] for i in range(1,len(inp))])

    node_info_set = [-1 for i in range(N)]
    for i in range(I):
        info_set.append([int(i) for i in f.readline().split()])
        for j in info_set[i] : node_info_set[j] = i

    nature_node = [-1 for i in range(N)]
    nature_transition = [[] for i in range(Na)]
    for i in range(Na):
        x, c = [int(i) for i in f.readline().split()]
        nature_node[x] = i
        for j in range(c):
            nature_transition[i].append([float(i) for i in f.readline().split()])
        

In [3]:
from prettytable import PrettyTable as pt

def preprocess():
    # utility of each leaf must have size P
    for u in utility:
        if len(u) < P:
            print("Game format invlaid")
            return -1
        
    # all nodes in an information set must belong to a player and have same strategies
    info_set_player = []
    info_set_strategies = []
    for i in range(I):
        player = node_master[info_set[i][0]]
        strategies = [c[1] for c in child[info_set[i][0]]]
        error = 0
        info_set_player.append(player)
        info_set_strategies.append(strategies)
        
    return info_set_player, info_set_strategies

def set_utility(info_set_s):
    re = compute_utility(info_set_s, 0)
    for i in range(P): result[i].append(re[i])

def compute_utility(info_set_s, c):
    if leaf_node[c] != -1 : return np.array(utility[leaf_node[c]])
    if nature_node[c] != -1 :
        re = np.array(0)
        for i in nature_transition[nature_node[c]]:
            re = re + compute_utility(info_set_s, int(i[0])) * i[1]
        return re
    x = info_set_s[node_info_set[c]]
    for i in child[c]:
        if(i[1] == x): return compute_utility(info_set_s, i[0])

    
def create_header(p, info_set_s):
    re = ''
    for i in range(I):
        if(info_set_player[i] == p):
            re += info_set_s[i]
    header[p].append(re)
    
def recursive_info_set_initial(p, i, info_set_s):
    while 1:
        if(i == I):
            if(p != P):
                if done[p] == 0 : create_header(p, info_set_s)
                recursive_info_set_initial(p+1, 0, info_set_s)
            else: 
                set_utility(info_set_s)
            return
        if(info_set_player[i] == p):
            if first[p] == -1 : first[p] = i
            for st in info_set_strategies[i]:
                info_set_s[i] = st
                recursive_info_set_initial(p, i+1, info_set_s)
            if first[p] == i : done[p] = 1
            return
        i += 1
    

def normal_form_converter():
    global result, header, done, first, count
    count = 0
    result = [[] for i in range(P)]
    header = [[] for i in range(P)]
    done = [0 for i in range(P)]
    first = [-1 for i in range(P)]
    recursive_info_set_initial(0, 0, ['' for i in range(I)])

    if(P == 2):
        for i in range(P):
            print('Player',i+1,'payoffs')
            table = pt()
            h = header[1].copy()
            h.insert(0, '\\')
            table.field_names = h
            for j in range(len(header[0])):
                temp = [header[0][j]]
                for k in range(j*(len(header[1])), (j+1)*(len(header[1]))): temp.append(result[i][k])
                table.add_row(temp)
            print(table)
    print('..........................')
    print_n_player(0,[0 for i in range(P)])
    
def print_n_player(p, arr):
    global count
    if p == P:
        for i in range(P):
            print('#'+str(i+1)+':', header[i][arr[i]-1], end=' ')
        print()
        for i in range(P):
            print('u'+str(i+1)+':', result[i][count], end=' ')
        print()
        count = count+1
        return
    for i in range(len(header[p])):
        arr[p] += 1
        print_n_player(p+1, arr)
    arr[p] = 0
        

In [4]:
def subgame_perfect(node):
    #print("node:" , node)
    if leaf_node[node] != -1: 
        return([utility[leaf_node[node]]])
    r = []
    if nature_node[node] != -1:
        for t in nature_transition[nature_node[node]]:
            r.extend(np.array(subgame_perfect(int(t[0])))*t[1])
        return r
    player = node_master[node]
    for c in child[node]:
        r.extend(subgame_perfect(c[0]))
    maxx = r[0][player]
    for i in r:
        print(i[player])
        maxx = max(maxx, i[player])
    result = []
    for i in r:
        if i[player] == maxx : result.append(i)
    return result

## Example 1
<img src="Images/3.png">

In [5]:
read_input('Test_Cases/test3.txt')
info_set_player, info_set_strategies = preprocess()
normal_form_converter()

Player 1 payoffs
+---+-----+-----+
| \ |  a  |  b  |
+---+-----+-----+
| A | 1.5 | 1.5 |
| B |  1  |  0  |
| C |  0  |  1  |
+---+-----+-----+
Player 2 payoffs
+---+-----+-----+
| \ |  a  |  b  |
+---+-----+-----+
| A | 3.5 | 3.5 |
| B |  2  |  0  |
| C |  0  |  0  |
+---+-----+-----+
..........................
#1: A #2: a 
u1: 1.5 u2: 3.5 
#1: A #2: b 
u1: 1.5 u2: 3.5 
#1: B #2: a 
u1: 1 u2: 2 
#1: B #2: b 
u1: 0 u2: 0 
#1: C #2: a 
u1: 0 u2: 0 
#1: C #2: b 
u1: 1 u2: 0 


## Example 2
<img src="Images/2.png">

In [11]:
read_input('Test_Cases/test2.txt')
info_set_player, info_set_strategies = preprocess()
normal_form_converter()

Player 1 payoffs
+----+----+----+----+----+
| \  | CE | CF | DE | DF |
+----+----+----+----+----+
| AG | 5  | 5  | 8  | 8  |
| AH | 5  | 5  | 8  | 8  |
| BG | 5  | 2  | 5  | 2  |
| BH | 5  | 1  | 5  | 1  |
+----+----+----+----+----+
Player 2 payoffs
+----+----+----+----+----+
| \  | CE | CF | DE | DF |
+----+----+----+----+----+
| AG | 4  | 4  | 3  | 3  |
| AH | 4  | 4  | 3  | 3  |
| BG | 5  | 1  | 5  | 1  |
| BH | 5  | 0  | 5  | 0  |
+----+----+----+----+----+
..........................
#1: AG #2: CE 
u1: 5 u2: 4 
#1: AG #2: CF 
u1: 5 u2: 4 
#1: AG #2: DE 
u1: 8 u2: 3 
#1: AG #2: DF 
u1: 8 u2: 3 
#1: AH #2: CE 
u1: 5 u2: 4 
#1: AH #2: CF 
u1: 5 u2: 4 
#1: AH #2: DE 
u1: 8 u2: 3 
#1: AH #2: DF 
u1: 8 u2: 3 
#1: BG #2: CE 
u1: 5 u2: 5 
#1: BG #2: CF 
u1: 2 u2: 1 
#1: BG #2: DE 
u1: 5 u2: 5 
#1: BG #2: DF 
u1: 2 u2: 1 
#1: BH #2: CE 
u1: 5 u2: 5 
#1: BH #2: CF 
u1: 1 u2: 0 
#1: BH #2: DE 
u1: 5 u2: 5 
#1: BH #2: DF 
u1: 1 u2: 0 


In [12]:
subgame_perfect(0)

[[5, 4], [5, 5]]